# Item 34: Avoid Injecting Data Into Generators with `send`

`yield` expressions provide generator functinons with a way to produce an iterable series of output values. But, this channel appears to be unidirectional: There's no immediately obvious way to stream data in and out of a generator as it runs. Having this could valuable for various use cases.

In [1]:
# Program to transmit signals using software-defined radio. The function generates an approximation of sine waves
import math

def wave(amplitue, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitue * fraction
        yield output

In [3]:
# Now, we can transmit the wave signal at a single specified amplitude by iterating over the wave generator
def transmit(output):
    if output is None:
        print('Output is None')
    else:
        print(f'Output: {output:>5.1f}')

def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))

Output:   0.0
Output:   2.1
Output:   3.0
Output:   2.1
Output:   0.0
Output:  -2.1
Output:  -3.0
Output:  -2.1


The code above works fine for producing basic waveforms, but it can't be used to constantly vary the amplitude of the wave based on a separate input. We need a way to modify the amplitude in each iteration of the generator.

Python generatos support the `send` method, which upgrades `yield` expressions into a two-way channel. The `send` method can be used to provide streaming inputs to a generator at the same, time it's yielding outputs. Normally, when iterating a generator, the value of the `yield` expression is `None`:

In [4]:
def my_generator():
    received = yield 1
    print(f'received = {received}')

it = iter(my_generator())
output = next(it)
print(f'output = {output}')

try:
    next(it) # Run generator until it exists
except StopIteration:
    pass

output = 1
received = None


When we call the `send` method instead of iterating the generator with a `for` loop or the `next` built-in function, the supplied parameter becomes the value of the `yield` expression when the generator is resumed. However, when the generator first starts, a `yield` expression has not been encountered yet, so the only valid value for calling `send` initially is `None` (any other argument would raise an exception at runtime):

In [5]:
it = iter(my_generator())
output = it.send(None) # Get first generator output
print(f'output = {output}')

try:
    it.send('hello!') # Send value into the generator
except StopIteration:
    pass

output = 1
received = hello!


In [6]:
# We can take advante of this behavior in order to modulate the amplitude of the sine wave based on an input
# signal. First, we need to change the wave generator to save the amplitude returned by the yield expression and
# use it to calulate the next generated output
def wave_modulating(steps):
    step_size = 2 * math.pi / steps
    amplitude = yield # Receive initial amplitude
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output # Receive next amplitude

In [7]:
# Then we update the run function to stream the modulating amplitude into the wave_modulating generator on each
# iteration. The first output to send must be None, since a yield expression would not have occurred within
# a generator yet
def run_modulating(it):
    amplitudes = [None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    for amplitude in amplitudes:
        output = it.send(amplitude)
        transmit(output)

run_modulating(wave_modulating(12))

Output is None
Output:   0.0
Output:   3.5
Output:   6.1
Output:   2.0
Output:   1.7
Output:   1.0
Output:   0.0
Output:  -5.0
Output:  -8.7
Output: -10.0
Output:  -8.7
Output:  -5.0


The above code has one problem, it is difficult for new readers to understand: `yield` expression on the right side, its hard knowing the connection between `yield` and `send` without previous knowledge of both.

In [8]:
# Requirements changed, now instead of using a single waveform, we need to use a complex waveform consisting of
# multiple signals in sequence. We can implement this by composing multiple generators together with yield from
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)

run(complex_wave())

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5


`yield from` expression handles the simpler case, so expect it to alos work properly along with the generator `send` method.

In [10]:
def complex_wave_modulating():
    yield from wave_modulating(3)
    yield from wave_modulating(4)
    yield from wave_modulating(5)

run_modulating(complex_wave_modulating())

Output is None
Output:   0.0
Output:   6.1
Output:  -6.1
Output is None
Output:   0.0
Output:   2.0
Output:   0.0
Output: -10.0
Output is None
Output:   0.0
Output:   9.5
Output:   5.9


Works but we have `None` values in the output. This happens because when each `yield from` expression finishes iterating over a nested generator, it moves on to the next one. Each nested generator starts with a bare `yield` expression-one without a value-in order to receive the initial amplitude from a generator `send` method call. This causes the parent generator to output a `None` value when it transitions between child generators.

This means that assumptions about how the `yield from` and `send` features behave individually will be broken of you try to use them together. 

The author recommends to avoid the `send` method entirely and go with a simpler approach.

In [14]:
# A simple solution to the above proble is to pass an iterator into the wave function. The iterator should 
# return an input amplitude each time the next function is called on it. This arrangement ensures that each
# generator is progressed in a cascade as inputs and outputs are processed
def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        amplitude = next(amplitude_it) # Get next input
        output = amplitude * fraction
        yield output

In [15]:
# We can then pass the same iterator into each of the generator functions that we're trying to compose together
def complex_wave_cascading(amplitude_it):
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)

In [18]:
# Now we can run the composed generator by passing in an iterator from the amplitudes list
def run_cascading():
    amplitudes = [7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    it = complex_wave_cascading(iter(amplitudes))
    for amplitude in amplitudes:
        output = next(it)
        transmit(output)

run_cascading()

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5
